In [17]:
import pandas as pd
import nltk
import ast
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sympy import sympify
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from pycaret.classification import *
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [18]:
sub_cnt = 50
#change sub_cnt to 10000 or other big value to use all training data
embedding_columns = ['Statement']
ignore_features = ['title']

In [19]:
data = pd.read_csv('../data/train_data.csv')
#data = pd.read_csv('../data/50%_train_data.csv')
data = data[:sub_cnt]

text_columns = ['title', 'Input', 'Output', 'Note', 'Statement']
text_list_columns = ['sample-input', 'sample-output']
text_features = text_columns + text_list_columns

In [20]:
def basic_preprocess(df):
    df[text_columns] = df[text_columns].fillna('missing')
    df[text_list_columns] = df[text_list_columns].fillna('[]')
    df['title'] = df['title'].str.extract(r'^[A-Za-z0-9]+\.\s*(.+)')
    df['TL'] = df['TL'].str.extract(r'(\d+)').astype(int)
    df['ML'] = df['ML'].str.extract(r'(\d+)').astype(int)
    df['tags'] = df['tags'].apply(ast.literal_eval)
    df['sample-input'] = df['sample-input'].apply(ast.literal_eval).apply(lambda x: " ".join(x))
    df['sample-output'] = df['sample-output'].apply(ast.literal_eval).apply(lambda x: " ".join(x))
    df.drop(columns='Unnamed: 0', inplace=True)
    df.drop(columns='contest-name', inplace=True)
    return df

data = basic_preprocess(data)
print(data.iloc[0])

title                                                   Squid Game
TL                                                               2
ML                                                             256
Input            The first line contains $$$2$$$ integer $$$n$$...
Output           Print the minimum number of operations Mashtal...
Note             Explanation for the first sample:  In the firs...
Statement        After watching the new over-rated series Squid...
contest                                                       1610
index                                                            H
tags             [data structures, dfs and similar, greedy, trees]
rating                                                      3500.0
sample-input     \n6 3\n1 1 1 4 4\n1 5\n3 4\n2 6\n \n5 3\n1 1 3...
sample-output                                         \n2\n \n-1\n
Name: 0, dtype: object


In [21]:
def text_preprocess(df):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    def text_transform(text):
        tokens = word_tokenize(text.lower())
        filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
        stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]
        return ' '.join(lemmatized_tokens)
    for feature in text_features:
        df[feature] = df[feature].apply(text_transform)
    return df

data = text_preprocess(data)
print(data.iloc[0])

title                                                   squid game
TL                                                               2
ML                                                             256
Input            first line contain 2 integ n 1 n 3 number vert...
Output           print minimum number oper mashtali way mashtal...
Note             explan first sampl first oper mashtali choos v...
Statement        watch new seri squid game mashtali soroush dec...
contest                                                       1610
index                                                            H
tags             [data structures, dfs and similar, greedy, trees]
rating                                                      3500.0
sample-input     6 3 1 1 1 4 4 1 5 3 4 2 6 5 3 1 1 3 3 1 2 1 4 1 5
sample-output                                                    2
Name: 0, dtype: object


In [22]:
def text_embedding(df):
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    #model_name = "sentence-transformers/all-mpnet-base-v2"
    #model_name = "sentence-transformers/all-roberta-large-v1" #worst
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    def generate_embeddings(text):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        outputs = model(**inputs)
        #return outputs.pooler_output.detach().numpy().flatten() #0.7320
        return torch.mean(outputs.last_hidden_state, dim=1).detach().numpy().flatten() #0.7360
    for feature in embedding_columns:
        df[feature] = df[feature].apply(generate_embeddings)
        embedding_df = pd.DataFrame(df[feature].to_list(), columns=[f'{feature}_Emb_{i}' for i in range(len(df[feature][0]))])
        df = pd.concat([df, embedding_df], axis=1)

    df = df.drop(columns=embedding_columns)
    return df
    
data = text_embedding(data)
print(data.iloc[0])

title                                                       squid game
TL                                                                   2
ML                                                                 256
Input                first line contain 2 integ n 1 n 3 number vert...
Output               print minimum number oper mashtali way mashtal...
                                           ...                        
Statement_Emb_379                                             0.021034
Statement_Emb_380                                             0.157798
Statement_Emb_381                                            -0.061132
Statement_Emb_382                                             -0.09207
Statement_Emb_383                                             0.066913
Name: 0, Length: 396, dtype: object


In [23]:
def tag_labeling(df):
    mlb = MultiLabelBinarizer()
    tags_binarized = mlb.fit_transform(df['tags'])
    tags_df = pd.DataFrame(tags_binarized, columns=mlb.classes_)
    df = pd.concat([df, tags_df], axis=1)
    df.drop(columns='tags', inplace=True)
    return tags_df, df

tags_df, data = tag_labeling(data)
print(data.iloc[0])

title                                                   squid game
TL                                                               2
ML                                                             256
Input            first line contain 2 integ n 1 n 3 number vert...
Output           print minimum number oper mashtali way mashtal...
                                       ...                        
probabilities                                                    0
sortings                                                         0
strings                                                          0
trees                                                            1
two pointers                                                     0
Name: 0, Length: 423, dtype: object


In [24]:
#preparing test data
test = pd.read_csv('../data/test_data.csv')
#test = pd.read_csv('../data/50%_test_data.csv')
test = basic_preprocess(test)
test = text_preprocess(test)
test = text_embedding(test)
tag, test = tag_labeling(test)

In [27]:
for tag in ['math', 'greedy', 'implementation', 'dp', 'data structures']:
    print(f'computing {tag}...')
    ignored_features = list(tags_df.columns.difference([tag]))
    exp = setup(data=data, 
                target=tag, 
                text_features=['sample-input', 'sample-output', 'Input', 'Output', 'Note'],
                ignore_features=ignored_features + ignore_features,
                fix_imbalance=True,
                session_id=123,
                )
    model = create_model('lightgbm')
    evaluate_model(model)
    predictions = predict_model(model, data=test)
    print("\n\n")

computing math...


,Description,Value
0,Session id,123
1,Target,math
2,Target type,Binary
3,Original data shape,"(50, 423)"
4,Transformed data shape,"(65, 1215)"
5,Transformed train set shape,"(50, 1215)"
6,Transformed test set shape,"(15, 1215)"
7,Ignore features,28
8,Numeric features,388
9,Text features,5


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5000,0.3333,0.0000,0.0000,0.0000,-0.3333,-0.3333
1,0.5000,0.6667,0.0000,0.0000,0.0000,-0.3333,-0.3333
2,0.5000,0.0000,0.0000,0.0000,0.0000,-0.3333,-0.3333
3,0.0000,0.0000,0.0000,0.0000,0.0000,-0.6000,-1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,0.6667,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.6667,0.5000,1.0000,0.5000,0.6667,0.4000,0.5000
7,0.6667,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.6667,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.6762,0.6073,0.3029,0.4334,0.3566,0.1492,0.1535


...




computing data structures...


,Description,Value
0,Session id,123
1,Target,data structures
2,Target type,Binary
3,Original data shape,"(50, 423)"
4,Transformed data shape,"(69, 1296)"
5,Transformed train set shape,"(54, 1296)"
6,Transformed test set shape,"(15, 1296)"
7,Ignore features,28
8,Numeric features,388
9,Text features,5


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5000,0.0000,0.0000,0.0000,0.0000,-0.3333,-0.3333
1,0.7500,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.5000,0.6667,0.0000,0.0000,0.0000,-0.3333,-0.3333
3,0.5000,0.6667,0.0000,0.0000,0.0000,-0.3333,-0.3333
4,0.5000,0.6667,0.0000,0.0000,0.0000,-0.3333,-0.3333
5,0.6667,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.6667,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.3333,0.5000,0.0000,0.0000,0.0000,-0.5000,-0.5000
8,1.0000,0.0000,0.0000,0.0000,0.0000,nan,0.0000


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.7855,0.5838,0.1912,0.2982,0.2330,0.1151,0.1190


In [9]:
test = pd.read_csv('../data/test_data.csv')
#test = pd.read_csv('../data/50%_test_data.csv')
test = basic_preprocess(test)
test = text_preprocess(test)
test = text_embedding(test)
tag, test = tag_labeling(test)

In [10]:
predictions = predict_model(model, data=test)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.7875,0.7571,0.2996,0.5620,0.3908,0.2763,0.2964


In [11]:
y_pred = predictions['prediction_label']
test['Predicted'] = y_pred
wrong_predictions = test[test['dp'] != test['Predicted']]
wrong_predictions.head()

,title,TL,ML,Input,Output,Note,contest,index,rating,sample-input,...,probabilities,schedules,shortest paths,sortings,string suffix structures,strings,ternary search,trees,two pointers,Predicted
1,submarin rybinsk sea easi edit,2,256,first line input contain singl integ n 1 n num...,print answer modulo,miss,1195,D1,1000.0,3 12 33 45 2 123 456 1 1 5 1000000000 10000000...,...,0,0,0,0,0,0,0,0,0,1
3,last minut enhanc,1,256,input consist multipl test case first line con...,test case output singl line contain precis one...,first test case euterp increas second fifth si...,1466,B,750.0,5 6 1 2 2 2 5 6 2 4 4 6 1 1 3 4 4 5 1 1 6 1 1 ...,...,0,0,0,0,0,0,0,0,0,0
4,minimum number variabl,1,256,first line contain integ n 1 n 23 second line ...,singl line print singl number minimum number v...,first sampl use two variabl b1 b2 perform foll...,279,D,2000.0,51 2 3 6 8 33 6 5 62 4 8 6 10 18,...,0,0,0,0,0,0,0,0,0,0
9,staircas,2,256,first line contain three integ n q 1 n 1000 1 ...,print q integ valu equal number differ stairca...,miss,1598,E,NaN,2 2 8 1 1 1 1 1 1 2 2 1 1 1 2 2 1 1 1 3 4 10 1...,...,0,0,0,0,0,0,0,0,0,0
18,short permut problem,7,1024,input consist singl line contain two integ n x...,let k answer pair k modulo k 0 output singl li...,first test case answer k shown follow tabl k 0...,1909,I,6000.0,3 2 4 1000000000 8 327869541 4000 1149333,...,0,0,0,0,0,0,0,0,0,0
